In [2]:
import array, csv, json, math
from osgeo import ogr
from random import shuffle, uniform
from shapely.geometry import shape, Point, Polygon

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromPolygon(geom):
    polygon = shape(geom)
    bbox = polygon.bounds
    l,b,r,t = bbox
    while True:
        point = Point(uniform(l,r),uniform(t,b))
        if point is None:
            break
        if polygon.contains(point):
            break
    return point.__geo_interface__['coordinates']

In [ ]:
# Transform NI Small Areas shapefile into GeoJSON
cmd = "ogr2ogr -f GeoJSON -t_srs crs:84 SA2011/SA2011.geojson SA2011/SA2011.shp"
!$cmd

In [7]:
# Load small areas GeoJSON file
with open("SA2011/SA2011.geojson") as f:
    sa2011 = json.load(f)

In [8]:
#sa2011['features'][0]['properties']
#{u'Hectares': 169.044,
# u'SA2011': u'N00000002',
# u'SOA2011': u'95AA01S2',
# u'X_COORD': 315465.0,
# u'Y_COORD': 377176.0}

In [9]:
belfastData = []
with open("Geographic Data (statistical geographies).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['HSCT'] == "Belfast HSCT":
            belfastData.append(row)

In [10]:
#belfastData[0]
#{'AA1998': 'BELFAST WEST',
# 'AA2008': 'BELFAST WEST',
# 'HSCT': 'Belfast HSCT',
# 'LGD1992NAME': 'BELFAST',
# 'SA': 'N00000825',
# 'SOA': '95GG01S1',
# 'SQ Km': '0.11',
# 'Settlement2015(1)': 'BELFAST CITY',
# 'Settlement2015(2)': '',
# 'Settlement2015(3)': '',
# 'URBAN_RURAL(2015)': 'Urban',
# 'WARD1992': 'ANDERSONSTOWN'}

In [11]:
belfastGeoJSON = { 
    "type": "FeatureCollection",
    "features": []
}
for row in belfastData:
    sa = row['SA']
    for feature in sa2011['features']:
        if feature['properties']['SA2011'] == sa:
            for key in row.keys():
                feature['properties'][key] = row[key]
            belfastGeoJSON['features'].append(feature)
            break
with open('belfast.geojson', 'w') as outfile:  
    json.dump(belfastGeoJSON, outfile, indent=4)

In [12]:
belfastSa2Idx = {}
for i in range(0,len(belfastGeoJSON["features"])):
    feature = belfastGeoJSON["features"][i]
    sa = feature['properties']['SA']
    belfastSa2Idx[sa] = i
len(belfastSa2Idx)

1008

In [13]:
religionData = []
with open("_KS211NI (s).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA Code'] in belfastSa2Idx:
            religionData.append(row)
len(religionData)

1008

In [14]:
#religionData[0]
#{'All usual residents': '367',
# 'Religion: Catholic': '320',
# 'Religion: Catholic (%)': '87.19',
# 'Religion: Church of Ireland': '0',
# 'Religion: Church of Ireland (%)': '0.00',
# 'Religion: Methodist Church in Ireland': '0',
# 'Religion: Methodist Church in Ireland (%)': '0.00',
# 'Religion: No religion': '15',
# 'Religion: No religion (%)': '4.09',
# 'Religion: Other Christian (including Christian related)': '3',
# 'Religion: Other Christian (including Christian related) (%)': '0.82',
# 'Religion: Other religions': '0',
# 'Religion: Other religions (%)': '0.00',
# 'Religion: Presbyterian Church in Ireland': '2',
# 'Religion: Presbyterian Church in Ireland (%)': '0.54',
# 'Religion: Religion not stated': '27',
# 'Religion: Religion not stated (%)': '7.36',
# 'SA': 'N00000825 (Andersonstown ward)',
# 'SA Code': 'N00000825'}

In [47]:
romanCatholic = ['Religion: Catholic']
presbyterian = ['Religion: Presbyterian Church in Ireland']
churchOfIreland = ['Religion: Church of Ireland']
methodist = ['Religion: Methodist Church in Ireland']
other = ['Religion: No religion', 'Religion: Other Christian (including Christian related)', 
         'Religion: Other religions', 'Religion: Religion not stated']

religions = {
    'romanCatholic':romanCatholic, 
    'presbyterian': presbyterian, 
    'churchOfIreland': churchOfIreland, 
    'methodist': methodist, 
    'other': other}

religion2Idx = {
    'romanCatholic': 1.0, 
    'presbyterian': 2.0, 
    'churchOfIreland': 3.0, 
    'methodist': 4.0, 
    'other': 5.0
}

In [32]:
geom = belfastGeoJSON["features"][0]["geometry"]
p = RandomPointFromPolygon(geom)
LonLatToPixelXY(p)

[123.73697328436603, 81.50378179192757]

In [62]:
points_2011 = []
for row in religionData:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2011.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [63]:
len(points_2011)

348204

In [72]:
shuffle(points_2011)

In [74]:
points = []
for sublist in points_2011:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2011.bin', 'wb'))

In [79]:
lookupTableData = []
with open("Look-up Tables_0.xlsx - SA2011.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA2011'] in belfastSa2Idx:
            lookupTableData.append(row)
len(lookupTableData)    

1008

In [81]:
#lookupTableData[0]
#{'COA2001_1': '95GG010001',
#'COA2001_2': '',
# 'COA2001_3': '',
# 'COA2001_4': '',
# 'SA2011': 'N00000825',
# 'SA2011NAME': 'N00000825 (Andersonstown ward)',
# 'SOA2001': '95GG01S1',
# 'SOANAME': 'Andersonstown_1'}

In [85]:
raw_2001 = []
with open("2001/oa/TABLE CAS308 (OA).csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        raw_2001.append(row)


In [121]:
religionDataIdx_2001 = {}

for row in lookupTableData:
    idx_2001 = row['COA2001_1']
    religionDataIdx_2001[idx_2001] = []
    for raw_row in raw_2001:
        if len(raw_row) == 3:
            raw_idx = raw_row[0].strip()
            raw_key = raw_row[1]
            raw_value = raw_row[2]
            if raw_value == "-":
                raw_value = '0'
            if raw_idx == idx_2001:
                religionDataIdx_2001[idx_2001].append({raw_key: raw_value})
            

In [124]:
religionData_2001 = []
for row in lookupTableData:
    data_2001 = {}
    idx_2001 = row['COA2001_1']
    data_2001['SA Code'] = row['SA2011']
    data_2001['COA2001_1'] = idx_2001
    data_2001['All usual residents'] = religionDataIdx_2001[idx_2001][0]['All persons']
    data_2001['Religion: Catholic'] = religionDataIdx_2001[idx_2001][2]['Catholic']
    data_2001['Religion: Church of Ireland'] = religionDataIdx_2001[idx_2001][4]['Church of Ireland']
    data_2001['Religion: Methodist Church in Ireland'] = religionDataIdx_2001[idx_2001][5]['Methodist Church in Ireland']
    data_2001['Religion: Presbyterian Church in Ireland'] = religionDataIdx_2001[idx_2001][3]['Presbyterian Church in Ireland']
    data_2001['Religion: Other Christian (including Christian related)'] = religionDataIdx_2001[idx_2001][6]['Other Christian (including Christian related)']
    data_2001['Religion: Other religions'] = religionDataIdx_2001[idx_2001][7]['Other religions and philosophies']
    data_2001['Religion: No religion'] = religionDataIdx_2001[idx_2001][8]['No religion or religion not stated']
    data_2001['Religion: Religion not stated'] = '0'
    religionData_2001.append(data_2001)
    
    

In [125]:
religionData_2001[0]

{'All usual residents': '432',
 'COA2001_1': '95GG010001',
 'Religion: Catholic': '411',
 'Religion: Church of Ireland': '3',
 'Religion: Methodist Church in Ireland': '0',
 'Religion: No religion': '18',
 'Religion: Other Christian (including Christian related)': '0',
 'Religion: Other religions': '0',
 'Religion: Presbyterian Church in Ireland': '0',
 'Religion: Religion not stated': '0',
 'SA Code': 'N00000825'}

In [117]:
religionDataIdx_2001['95GG010002']

[{'All persons': '415'},
 {'': '*'},
 {'Catholic': '393'},
 {'Presbyterian Church in Ireland': 0},
 {'Church of Ireland': 0},
 {'Methodist Church in Ireland': 0},
 {'Other Christian (including Christian related)': 0},
 {'Other religions and philosophies': 0},
 {'No religion or religion not stated': '22'}]

In [115]:
data_2001

{'COA2001_1': '95GG010001', 'SA Code': 'N00000825'}

In [126]:
religionDataIdx_2001[idx_2001]

[{'All persons': '294'},
 {'': '*'},
 {'Catholic': '130'},
 {'Presbyterian Church in Ireland': '43'},
 {'Church of Ireland': '27'},
 {'Methodist Church in Ireland': '26'},
 {'Other Christian (including Christian related)': '13'},
 {'Other religions and philosophies': '0'},
 {'No religion or religion not stated': '55'}]

In [127]:
points_2001 = []
for row in religionData_2001:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2001.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [128]:
len(points_2001)

309518

In [129]:
shuffle(points_2001)

In [130]:
points = []
for sublist in points_2001:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2001.bin', 'wb'))

In [164]:
grid2OA = []
with open("5672/5672package/package/100mgrids/txt/grid100mOA.txt") as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        grid2OA.append(row)

In [182]:
#grid2OA[0]
#{'Grid01': 'J 075 899',
# 'Grid718191': 'J075899 - J076900',
# 'OA': '95AA050006',
# 'Oa_code': '6',
# 'Oa_cty': '95',
# 'Oa_dis': 'AA',
# 'Oa_ward': '5',
# 'Ward': '95AA05',
# 'X': '307550',
# 'Y': '389950'}

In [167]:
oa2Grid718191 = {}
for row in grid2OA:
    if not row["OA"] in oa2Grid718191:
        oa2Grid718191[row["OA"]] = []
    oa2Grid718191[row["OA"]].append(row['Grid718191'])

In [183]:
#oa2Grid718191["95GG400001"]
#['J321744 - J322745',
# 'J321745 - J322746',
# 'J321746 - J322747',
# 'J322745 - J323746',
# 'J322747 - J323748']

In [171]:
# Convert DBF to CSV
years = ["1971", "1981", "1991"]
for year in years:
    cmd = "ogr2ogr -f CSV %soth100m.csv 5672/5672package/package/100mgrids/dbf/%soth100m.dbf" % (year, year)
    !$cmd
   

In [218]:
data_1991 = []
with open("1991oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1991.append(row)
    

In [219]:
# data_1991[0]
# {'GRID': 'C414160 - C415161',
#   ...
# 'V783_91': '-99997',
# 'V784_91': '-99997',
# 'V785_91': '-99997',
# 'V786_91': '-99997',
# 'V787_91': '-99997',
# 'V788_91': '-99997',
# 'V789_91': '-99997',
# 'V790_91': '-99997',
# 'V791_91': '-99997',
# 'V792_91': '-99997',
# 'V793_91': '-99997',
# 'V794_91': '-99997',
# ...
# }

In [220]:
# 1971 and 1981
#     totalCatholic = int(row['V783_91']) + int(row['V784_91'])
#    totalPresbyterian = int(row['V785_91']) + int(row['V786_91'])
#    totalCoI = int(row['V787_91']) + int(row['V788_91'])
#    totalMethodist = int(row['V789_91']) + int(row['V790_91'])
#    totalOther = int(row['V791_91']) + int(row['V792_91'])    
#    totalNotStated = int(row['V793_91']) + int(row['V794_91'])

grid2religion_1991 = {}
for row in data_1991:
    gridIdx = row['GRID']
    totalCatholic = int(row['V895_91']) + int(row['V896_91'])
    totalPresbyterian = int(row['V897_91']) + int(row['V898_91'])
    totalCoI = int(row['V899_91']) + int(row['V900_91'])
    totalMethodist = int(row['V901_91']) + int(row['V902_91'])
    totalOther = int(row['V903_91']) + int(row['V904_91'])    
    totalNone = int(row['V905_91']) + int(row['V906_91'])
    totalNotStated = int(row['V907_91']) + int(row['V908_91'])
    grid2religion_1991[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated),
        'None': max(0,totalNone)
    }
    

In [221]:
religionData_1991 = []
for row in lookupTableData:
    data_1991 = {}
    idx_2001 = row['COA2001_1']
    data_1991['SA Code'] = row['SA2011']
    data_1991['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1991:
                totalCatholic += grid2religion_1991[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1991[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1991[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1991[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1991[gridIdx]['Other']
                totalNotStated += grid2religion_1991[gridIdx]['Religion not stated']
                totalNone += grid2religion_1991[gridIdx]['None']
        
    data_1991['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated + totalNone
    data_1991['Religion: Catholic'] = totalCatholic
    data_1991['Religion: Church of Ireland'] = totalCoI
    data_1991['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1991['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1991['Religion: Other Christian (including Christian related)'] = totalOther
    data_1991['Religion: Other religions'] = 0
    data_1991['Religion: No religion'] = totalNone
    data_1991['Religion: Religion not stated'] = totalNotStated
    religionData_1991.append(data_1991)


In [225]:
religionData_1991[1000]

{'All usual residents': 99,
 'COA2001_1': '95GG510008',
 'Religion: Catholic': 0,
 'Religion: Church of Ireland': 48,
 'Religion: Methodist Church in Ireland': 7,
 'Religion: No religion': 5,
 'Religion: Other Christian (including Christian related)': 3,
 'Religion: Other religions': 0,
 'Religion: Presbyterian Church in Ireland': 31,
 'Religion: Religion not stated': 10,
 'SA Code': 'N00001640'}

In [205]:
found = 0
missed = 0
for row in grid2OA:
    gridIdx = row['Grid718191']
    matched = False
    for val in data_1991:        
        if gridIdx == val['GRID']:
            matched = True
            break
    if matched:
        found += 1
    else:
        missed += 1 

In [206]:
missed

7128

In [226]:
found

18630

In [229]:
points_1991 = []
for row in religionData_1991:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1991.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

997


In [230]:
len(points_1991)

261815

In [231]:
shuffle(points_1991)
points = []
for sublist in points_1991:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1991.bin', 'wb'))

In [233]:
data_1981 = []
with open("1981oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1981.append(row)
    

# 1971 and 1981

grid2religion_1981 = {}
for row in data_1981:
    gridIdx = row['GRID']
    totalCatholic = int(row['V783_81']) + int(row['V784_81'])
    totalPresbyterian = int(row['V785_81']) + int(row['V786_81'])
    totalCoI = int(row['V787_81']) + int(row['V788_81'])
    totalMethodist = int(row['V789_81']) + int(row['V790_81'])
    totalOther = int(row['V781_81']) + int(row['V792_81'])    
    totalNotStated = int(row['V793_81']) + int(row['V794_81'])
    grid2religion_1981[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated)
    }
    

religionData_1981 = []
for row in lookupTableData:
    data_1981 = {}
    idx_2001 = row['COA2001_1']
    data_1981['SA Code'] = row['SA2011']
    data_1981['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1981:
                totalCatholic += grid2religion_1981[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1981[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1981[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1981[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1981[gridIdx]['Other']
                totalNotStated += grid2religion_1981[gridIdx]['Religion not stated']
                
    data_1981['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated 
    data_1981['Religion: Catholic'] = totalCatholic
    data_1981['Religion: Church of Ireland'] = totalCoI
    data_1981['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1981['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1981['Religion: Other Christian (including Christian related)'] = totalOther
    data_1981['Religion: Other religions'] = 0
    data_1981['Religion: No religion'] = totalNone
    data_1981['Religion: Religion not stated'] = totalNotStated
    religionData_1981.append(data_1981)

points_1981 = []
for row in religionData_1981:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1981.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

shuffle(points_1981)
points = []
for sublist in points_1981:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1981.bin', 'wb'))


997


In [234]:
len(points_1981)

272125

In [236]:
data_1971 = []
with open("1971oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1971.append(row)
    

# 1971 and 1971

grid2religion_1971 = {}
for row in data_1971:
    gridIdx = row['GRID']
    totalCatholic = int(row['V783_71']) + int(row['V784_71'])
    totalPresbyterian = int(row['V785_71']) + int(row['V786_71'])
    totalCoI = int(row['V787_71']) + int(row['V788_71'])
    totalMethodist = int(row['V789_71']) + int(row['V790_71'])
    totalOther = int(row['V771_71']) + int(row['V792_71'])    
    totalNotStated = int(row['V793_71']) + int(row['V794_71'])
    grid2religion_1971[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated)
    }


religionData_1971 = []
for row in lookupTableData:
    data_1971 = {}
    idx_2001 = row['COA2001_1']
    data_1971['SA Code'] = row['SA2011']
    data_1971['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1971:
                totalCatholic += grid2religion_1971[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1971[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1971[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1971[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1971[gridIdx]['Other']
                totalNotStated += grid2religion_1971[gridIdx]['Religion not stated']
                
    data_1971['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated 
    data_1971['Religion: Catholic'] = totalCatholic
    data_1971['Religion: Church of Ireland'] = totalCoI
    data_1971['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1971['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1971['Religion: Other Christian (including Christian related)'] = totalOther
    data_1971['Religion: Other religions'] = 0
    data_1971['Religion: No religion'] = totalNone
    data_1971['Religion: Religion not stated'] = totalNotStated
    religionData_1971.append(data_1971)

points_1971 = []
for row in religionData_1971:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1971.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

shuffle(points_1971)
points = []
for sublist in points_1971:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1971.bin', 'wb'))



997


In [237]:
len(points_1971)

362403